Dans le cadre de la sécurisation des espaces publics et privés, la détection automatique de personnes armées à partir de flux vidéo est un enjeu majeur. Les systèmes de détection basés sur l'intelligence artificielle peuvent offrir une surveillance proactive, capable d’identifier rapidement des individus portant des armes, permettant ainsi une intervention rapide pour minimiser les risques de violence.
Dans ce projet, nous avons développé un modèle de détection d’images de personnes armées. Pour constituer notre jeu de données, nous avons d'abord téléchargé un ensemble d’images et de vidéos sur Kaggle, une plateforme reconnue pour ses vastes collections de données. Les vidéos collectées ont été renommées selon le format x-y (où x représente les minutes et y les secondes) pour faciliter l’identification temporelle et l'organisation des données.
Ensuite, nous avons utilisé la bibliothèque OpenCV pour extraire des images spécifiques à partir des vidéos. Pour chaque séquence vidéo contenant un suspect, nous avons capturé 10 images avant que le suspect ne porte une arme, et 10 images pendant qu'il est en possession de celle-ci. Cette approche nous permet d’obtenir un ensemble de données équilibré entre des situations armées et non armées, essentiel pour entraîner et évaluer un modèle de détection d'images capable de différencier ces deux contextes.

In [22]:
import cv2
import os
import re
import pandas as pd

In [ ]:

# Chemin du dossier contenant les vidéos
video_folder = '/Users/kader/Downloads/Exploitable'

# Chemins des dossiers de sortie pour les frames avant et après
before_folder = os.path.join(video_folder, 'sans')
after_folder = os.path.join(video_folder, 'avec')

# Créer les dossiers "sans" et "avec" s'ils n'existent pas
os.makedirs(before_folder, exist_ok=True)
os.makedirs(after_folder, exist_ok=True)

# Fonction pour convertir le format "minutes-secondes" en secondes
def parse_time_from_filename(filename):
    match = re.search(r'(\d+)-(\d+)', filename)
    if match:
        minutes = int(match.group(1))
        seconds = int(match.group(2))
        return minutes * 60 + seconds
    else:
        raise ValueError("Format du nom de fichier incorrect. Assurez-vous qu'il contient un moment comme '2-08'.")

# Fonction principale pour extraire les images autour du moment donné
def extract_images_from_video(video_path, extraction_time, num_frames=10):
    # Charger la vidéo
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Erreur lors de l'ouverture de la vidéo : {video_path}")
        return
    
    # Obtenir les fps (images par seconde)
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Calculer l'index de la frame centrale
    frame_center = int(extraction_time * fps)
    
    # Créer une liste des indices des frames à extraire (10 avant et 10 après, et le moment central)
    frame_indices = [frame_center + offset for offset in range(-num_frames, num_frames + 1)]

    # Extraire et enregistrer les frames
    for i, frame_id in enumerate(frame_indices):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        ret, frame = cap.read()
        
        if ret:
            # Choisir le dossier de sortie en fonction de la position par rapport au moment central
            if frame_id < frame_center:
                output_folder = before_folder  # Frames avant le moment donné
            elif frame_id > frame_center:
                output_folder = after_folder   # Frames après le moment donné
            else:
                output_folder = video_folder   # Moment exact

            # Construire le nom de fichier de sortie
            base_filename = os.path.splitext(os.path.basename(video_path))[0]
            output_path = os.path.join(output_folder, f"{base_filename}_frame_{frame_id}.jpg")
            
            # Enregistrer l'image
            cv2.imwrite(output_path, frame)
            print(f"Image extraite et enregistrée à : {output_path}")
        else:
            print(f"Impossible de lire la frame {frame_id} pour la vidéo {video_path}")

    # Libérer les ressources
    cap.release()

# Parcourir chaque fichier vidéo dans le dossier
for filename in os.listdir(video_folder):
    if filename.endswith('.mp4'):
        # Obtenir le chemin complet du fichier vidéo
        video_path = os.path.join(video_folder, filename)
        
        try:
            # Extraire le temps d'extraction du nom du fichier
            extraction_time = parse_time_from_filename(filename)
            print(f"Traitement de {filename} - Extraction à {extraction_time} secondes")
            
            # Extraire les images autour du moment
            extract_images_from_video(video_path, extraction_time)
        except ValueError as e:
            print(e)

In [36]:
# Chemin du dossier principal contenant les sous-dossiers 'sans' et 'avec'
base_path = 'data'

# Initialisation de la liste pour stocker les données
data = []

# Parcourir les dossiers d'images
for label, folder in zip([0, 1], ['sans', 'avec']):
    folder_path = '/Users/kader/Downloads/Exploitable'
    
    # Vérifier si le dossier existe
    if os.path.exists(folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith(('.jpg', '.png')):  # Filtre pour les fichiers d'image
                file_path = os.path.join(folder_path, filename)
                data.append([file_path, label])  # Ajouter le chemin et le label (0 ou 1)

# Création du DataFrame avec les colonnes 'FilePath' et 'Label'
df = pd.DataFrame(data, columns=['FilePath', 'Label'])

# Affichage des 5 premières lignes du DataFrame
df

,FilePath,Label
0,/Users/kader/Downloads/Exploitable/Robbery075_...,0
1,/Users/kader/Downloads/Exploitable/Robbery087_...,0
2,/Users/kader/Downloads/Exploitable/Robbery086_...,0
3,/Users/kader/Downloads/Exploitable/Robbery100_...,0
4,/Users/kader/Downloads/Exploitable/Robbery079_...,0
5,/Users/kader/Downloads/Exploitable/Robbery071_...,0
6,/Users/kader/Downloads/Exploitable/Robbery084_...,0
7,/Users/kader/Downloads/Exploitable/Robbery078_...,0
8,/Users/kader/Downloads/Exploitable/Robbery094_...,0
9,/Users/kader/Downloads/Exploitable/Robbery073_...,0


In [4]:
# Ajouter des transformations d'augmentation des données
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,  # Rotation aléatoire de 20 degrés
    width_shift_range=0.2,  # Décalage horizontal aléatoire
    height_shift_range=0.2,  # Décalage vertical aléatoire
    shear_range=0.2,  # Cisaillement aléatoire
    zoom_range=0.2,  # Zoom aléatoire
    horizontal_flip=True,  # Retournement horizontal aléatoire
    fill_mode='nearest',  # Remplissage des pixels vides après transformation
    validation_split=0.2  # Fraction des données utilisée pour la validation
)

# Chargement des images d'entraînement avec augmentation
train_generator = datagen.flow_from_directory(
    'path_to_your_pokemon_dataset/',
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Chargement des images de validation (sans augmentation)
validation_generator = datagen.flow_from_directory(
    'path_to_your_pokemon_dataset/',
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)
